In [1]:
import pandas as pd
import optuna
import xgboost as xgb
import numpy as np

/home/dev/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
storm_data = pd.read_csv('storm_data.csv', parse_dates=True)
storm_data = storm_data.dropna()
# remove column limit in pandas
pd.set_option('display.max_columns', None)
storm_data.sample(10)

,date,index,ticker,S_prime,K,t_prime,S,volume,opt_price_prime,opt_price,best_bid,best_offer,volatility,rf,idx,V,V_prime,V_prime_delta,V_delta,BS_V,BS_V_prime
286411,2023-06-20,4128529,NOK,0.085990,45.0,17,3.869537,52,0.000556,0.025,0.02,0.03,0.016745,0.000139,18,0.087883,0.001953,0.005890,0.265049,0.0,0.0
340609,2023-08-17,5741922,GME,0.183800,100.0,8,18.379999,0,0.086500,8.650,7.90,9.40,0.035357,0.000145,0,0.136533,0.001365,0.003823,0.382310,0.0,0.0
336447,2023-08-14,4322053,PFE,0.069913,440.0,18,30.761602,0,0.000011,0.005,0.00,0.01,0.012444,0.000144,22,0.896044,0.002036,0.006233,2.742317,0.0,0.0
301132,2023-07-06,4095105,NKE,0.064697,1550.0,15,100.280853,0,0.000003,0.005,0.00,0.01,0.016968,0.000143,17,2.462604,0.001589,0.004491,6.960319,0.0,0.0
26304,2022-09-27,1118173,BB,0.050900,100.0,10,5.090000,0,0.000100,0.010,0.00,0.02,0.038487,0.000089,0,0.107968,0.001080,0.002640,0.264047,0.0,0.0
147996,2023-01-27,2641972,JPM,0.093353,1400.0,14,130.694824,135,0.001732,2.425,2.40,2.45,0.017347,0.000125,0,2.270555,0.001622,0.004640,6.495826,0.0,0.0
1658,2022-09-01,340330,XOM,0.074994,1120.0,1,83.993080,3,0.000013,0.015,0.00,0.03,0.024388,0.000078,0,1.001776,0.000894,0.002432,2.724071,0.0,0.0
256129,2023-05-17,5716586,GME,0.190583,120.0,16,22.870001,0,0.090417,10.850,10.45,11.25,0.055344,0.000139,30,0.301877,0.002516,0.008423,1.010764,0.0,0.0
129487,2023-01-09,4398647,BB,0.020353,170.0,11,3.460000,0,0.000029,0.005,0.00,0.01,0.033284,0.000142,0,0.181552,0.001068,0.002579,0.438396,0.0,0.0
255455,2023-05-17,3435495,MCD,0.096620,2850.0,16,275.368378,0,0.003474,9.900,9.75,10.05,0.008879,0.000139,10,5.312115,0.001864,0.005560,15.846046,0.0,0.0


In [3]:
# for each ticker we need the 60 day rolling volatility, first read the price data


vols = {}
for ticker in storm_data["ticker"].unique():
    price_data = pd.read_csv(f"options_data/{ticker}.csv", index_col=0, parse_dates=True)
    rolling_vol = price_data["Close"].pct_change().rolling(window=60).std().dropna()
    vols[ticker] = rolling_vol


In [4]:
def get_vol(row):
    ticker = row["ticker"]
    date = row["date"]
    return vols[ticker].loc[date]

storm_data["vol"] = storm_data.apply(get_vol, axis=1)

In [5]:
storm_data.sample(10)

,date,index,ticker,S_prime,K,t_prime,S,volume,opt_price_prime,opt_price,best_bid,best_offer,volatility,rf,idx,V,V_prime,V_prime_delta,V_delta,BS_V,BS_V_prime,vol
117878,2022-12-28,653205,MCD,0.130178,1900.0,23,247.338089,0,0.040039,76.075,75.65,76.50,0.011748,0.000119,11,6.736997,0.003546,0.012408,23.575045,0.0,0.0,0.011860
122036,2022-12-30,1856316,GME,0.078553,235.0,21,18.459999,28,0.001723,0.405,0.39,0.42,0.046012,0.000117,30,0.606374,0.002580,0.008302,1.951029,0.0,0.0,0.045923
297925,2023-07-03,2976195,XOM,0.080098,1230.0,11,98.520576,0,0.000012,0.015,0.00,0.03,0.016702,0.000141,0,1.529758,0.001244,0.003240,3.985323,0.0,0.0,0.014047
317572,2023-07-25,2981923,XOM,0.106471,910.0,3,96.888664,0,0.016456,14.975,14.90,15.05,0.016741,0.000144,0,0.983047,0.001080,0.002903,2.642174,0.0,0.0,0.015323
79441,2022-11-17,965319,NKE,0.103759,960.0,22,99.608482,4,0.010443,10.025,9.85,10.20,0.027587,0.000113,18,2.888194,0.003009,0.010022,9.621599,0.0,0.0,0.031501
234973,2023-04-26,4409223,BB,0.113429,35.0,23,3.970000,0,0.015857,0.555,0.50,0.61,0.035795,0.000137,27,0.117116,0.003346,0.011417,0.399603,0.0,0.0,0.034598
90004,2022-11-29,1845644,GME,0.059535,430.0,24,25.600000,0,0.000698,0.300,0.23,0.37,0.044964,0.000117,31,1.291895,0.003004,0.009848,4.234665,0.0,0.0,0.047255
219717,2023-04-11,5703956,GME,0.174538,130.0,3,22.690001,0,0.074615,9.700,9.10,10.30,0.059182,0.000134,0,0.169730,0.001306,0.003822,0.496872,0.0,0.0,0.064459
42195,2022-10-12,1120157,BB,0.210000,20.0,9,4.200000,0,0.111250,2.225,2.07,2.38,0.034443,0.000097,0,0.031120,0.001556,0.004585,0.091699,0.0,0.0,0.029530
86717,2022-11-25,1067102,PFE,0.096875,420.0,7,40.687534,0,0.017321,7.275,7.15,7.40,0.015059,0.000114,0,0.445069,0.001060,0.002707,1.137028,0.0,0.0,0.016134


In [6]:
# train test split 70/10/20
dates = storm_data["date"].unique()
train_dates = dates[:int(len(dates)*0.7)]
val_dates = dates[int(len(dates)*0.7):int(len(dates)*0.8)]
test_dates = dates[int(len(dates)*0.8):]

train_data = storm_data[storm_data["date"].isin(train_dates)]
val_data = storm_data[storm_data["date"].isin(val_dates)]
test_data = storm_data[storm_data["date"].isin(test_dates)]


In [7]:
# create an xgb dmatrix for train, val and test
input_cols = ["S", "K", "t_prime", "rf", "vol"]
output_col = "opt_price"

train_dmatrix = xgb.DMatrix(train_data[input_cols], label=train_data[output_col])
val_dmatrix = xgb.DMatrix(val_data[input_cols], label=val_data[output_col])
test_dmatrix = xgb.DMatrix(test_data[input_cols], label=test_data[output_col])


In [8]:
# define the objective function
def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "eta": trial.suggest_loguniform("eta", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.1, 10),
        "subsample": trial.suggest_uniform("subsample", 0.5, 1),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1),
        "gamma": trial.suggest_uniform("gamma", 0, 1),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 0.01, 10),   
    }
    model = xgb.train(params, train_dmatrix, num_boost_round=100, evals=[(val_dmatrix, "validation")], verbose_eval=False)
    preds = model.predict(val_dmatrix)
    rmse = np.sqrt(np.mean((preds - val_data[output_col]) ** 2))
    return rmse


In [9]:
# create a study and optimize the objective function
STUDY_NAME = "xgb_opt_price"
STUDY_LOCATION = "sqlite:///optuna_dbs/xgboost.db"

study = optuna.create_study(direction="minimize", 
                            study_name=STUDY_NAME,
                            storage=STUDY_LOCATION,
                            load_if_exists=True)

#find the number of completed trials
completed_trials = len([s for s in study.trials if s.state == optuna.trial.TrialState.COMPLETE])
n_trials = 100 - completed_trials
if n_trials > 0:
    study.optimize(objective, n_trials=n_trials)

# print the best parameters
print(study.best_params)


[I 2026-02-13 14:27:04,027] Using an existing study with name 'xgb_opt_price' instead of creating a new one.


{'eta': 0.20181179422303602, 'max_depth': 9, 'min_child_weight': 0.13594125663047535, 'subsample': 0.8955123693496504, 'colsample_bytree': 0.9529491841320068, 'gamma': 0.6535351385890895, 'reg_alpha': 1.8529104538359418}


In [10]:
# now train the best model on the train data
best_params = study.best_params
best_model = xgb.train(best_params, train_dmatrix, num_boost_round=100, evals=[(val_dmatrix, "validation")], verbose_eval=False)

# now predict the test data
test_preds = best_model.predict(test_dmatrix)

In [11]:
test_df = test_data.copy()
test_df["xgboost_v"] = test_preds
test_df["xgboost_v"] = test_df["xgboost_v"].clip(lower=0)
test_df["xgboost_v_prime"] = test_df["xgboost_v"] /test_df["K"]
test_df.sample(10)

,date,index,ticker,S_prime,K,t_prime,S,volume,opt_price_prime,opt_price,best_bid,best_offer,volatility,rf,idx,V,V_prime,V_prime_delta,V_delta,BS_V,BS_V_prime,vol,xgboost_v,xgboost_v_prime
324428,2023-08-01,3458410,MCD,0.100762,2725.0,17,274.575592,0,0.007165,19.525,19.40,19.65,0.007223,0.000142,10,5.556409,0.002039,0.006241,17.007807,0.0,0.0,0.007262,23.460949,0.008610
347300,2023-08-24,5744035,GME,0.155810,105.0,8,16.360001,0,0.060714,6.375,5.55,7.20,0.035691,0.000145,0,0.132678,0.001264,0.003410,0.358032,0.0,0.0,0.038543,3.621862,0.034494
323938,2023-08-01,2697330,JPM,0.105362,1410.0,3,148.560547,1,0.011596,16.350,16.25,16.45,0.016489,0.000142,0,1.516384,0.001075,0.002887,4.070033,0.0,0.0,0.010766,15.534749,0.011018
330579,2023-08-08,2985738,XOM,0.063721,1550.0,10,98.768120,0,0.000026,0.040,0.00,0.08,0.016078,0.000145,0,1.728770,0.001115,0.002774,4.299053,0.0,0.0,0.013655,2.295766,0.001481
308436,2023-07-14,2692452,JPM,0.104086,1360.0,7,141.556885,1,0.010184,13.850,13.75,13.95,0.016725,0.000143,0,1.475183,0.001085,0.002770,3.767819,0.0,0.0,0.011745,10.465538,0.007695
302440,2023-07-07,4095399,NKE,0.078480,1270.0,7,99.670197,0,0.000004,0.005,0.00,0.01,0.016974,0.000143,0,1.298358,0.001022,0.002566,3.259362,0.0,0.0,0.017259,0.000000,0.000000
347787,2023-08-25,2991001,XOM,0.097144,1030.0,21,100.058403,0,0.005995,6.175,6.10,6.25,0.014360,0.000146,6,2.823275,0.002741,0.009002,9.271636,0.0,0.0,0.013805,9.207473,0.008939
306071,2023-07-12,3451976,MCD,0.094528,2950.0,9,278.858307,894,0.000927,2.735,2.68,2.79,0.007649,0.000143,0,3.352658,0.001136,0.002883,8.505714,0.0,0.0,0.007369,2.372827,0.000804
333939,2023-08-10,4321499,PFE,0.060926,500.0,15,30.463112,0,0.000070,0.035,0.00,0.07,0.012546,0.000144,27,0.796970,0.001594,0.004538,2.268772,0.0,0.0,0.014493,0.740611,0.001481
317139,2023-07-24,5734421,GME,0.148645,155.0,4,23.040001,0,0.045484,7.050,6.25,7.85,0.051483,0.000144,0,0.183744,0.001185,0.003273,0.507316,0.0,0.0,0.041715,6.490413,0.041874


In [15]:
from scipy.stats import wasserstein_distance

market_col = "opt_price"
bs_col = "BS_V"
qstorm_col = "V"
xgb_col = "xgboost_v"

# Store results per ticker in a nested dict
results = {}
tickers = test_df["ticker"].unique()
for ticker in tickers:
    ticker_df = test_df[test_df["ticker"] == ticker]
    rmse = {}
    wass = {}
    for col, name in zip([qstorm_col, bs_col, xgb_col], ["qStorm", "BSM", "xgb"]):
        error = ticker_df[market_col] - ticker_df[col]
        squared_error = error**2
        rmse[name] = np.sqrt(squared_error.mean())
        wass[name] = wasserstein_distance(ticker_df[market_col], ticker_df[col])
    results[ticker] = {"RMSE": rmse, "Wasserstein": wass}

# Transform results into a pandas DataFrame for display
result_df = pd.concat(
    {
        ticker: pd.DataFrame({
            "RMSE": results[ticker]["RMSE"],
            "Wasserstein": results[ticker]["Wasserstein"]
        }) for ticker in results
    },
    names=["Ticker", "Metric"]
)

# Add a "total" summary column (average over tickers) for each method/metric
# First, unstack to get methods as columns, then calculate mean per method
result_df_unstacked = result_df.unstack(level="Metric")  # Ticker x (Method x Metric)
# Compute averages for each metric, across tickers, for each method
total_row = result_df_unstacked.mean(axis=0)
# Append total row
result_df_with_total = pd.concat(
    [result_df_unstacked, total_row.to_frame(name='total').T]
)
result_df_with_total = np.round(result_df_with_total, 4)
result_df_with_total


RMSE                   Wasserstein                 
Metric   qStorm      BSM      xgb      qStorm      BSM     xgb
JPM     27.2532  28.6770  10.8235     16.1813  18.3619  5.7506
XOM     13.7444  14.5863   1.8275      6.8925   8.1086  1.3021
MCD     50.4632  53.1928   2.5808     29.8711  33.1601  1.3794
NKE     12.1934  13.0218   3.5549      5.9181   7.1287  2.7199
NOK      1.4587   1.5072   0.1942      0.8475   0.9065  0.0790
PFE      3.7437   4.0143   1.5878      1.9304   2.2279  1.2006
BB       1.7361   1.8202   0.1122      1.0206   1.1228  0.0377
GME      4.7259   4.9167   1.0797      2.3794   2.7545  0.4181
total   14.4148  15.2171   2.7201      8.1301   9.2214  1.6109